In [ ]:
import os
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import sys
from matplotlib import animation, rc
from IPython.display import HTML, Image
import pandas as pd

src_dir = os.path.abspath("/home/mmfarrugia/repos/optimization/hybrid_optimizer")
sys.path.append(src_dir)

from hybrid_optimizer import PSO_DE
from pyswarms.utils.plotters.formatters import Mesher, Designer
from pyswarms.utils.plotters.plotters import plot_cost_history, plot_contour
from plotters import plot_summary, plot_surface
import example_funcs

technical_replicates = 10
size_pop = 50
max_iter = 200

In [ ]:
# Universal config setup
random_config = {
    'name': 'Random',
    "n_dim": 2,
    "size_pop": size_pop,
    "vectorize_func": False,
    "taper_DE": False,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/rand/1'
}
random_fixedF_config = {
    'name': 'RandomFixedF',
    "n_dim": 2,
    "size_pop": size_pop,
    "vectorize_func": False,
    "F": (0.5, 0.5),
    "taper_DE": False,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/rand/1'
}
best_config = {
    'name': 'Best',
    "n_dim": 2,
    "size_pop": size_pop,
    "vectorize_func": False,
    "taper_DE": False,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/best/1'
}
best_fixedF_config = {
    'name': 'BestFixedF',
    "n_dim": 2,
    "F": (0.5, 0.5),
    "size_pop": size_pop,
    "vectorize_func": False,
    "taper_DE": False,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/best/1'
}

taper_rand_config = {
    'name': 'RandTaperFreq',
    "n_dim": 2,
    "size_pop": size_pop,
    "vectorize_func": False,
    "taper_DE": True,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/rand/1'
}

taper_best_config = {
    'name': 'BestTaperFreq',
    "n_dim": 2,
    "size_pop": size_pop,
    "vectorize_func": False,
    "taper_DE": True,
    "max_iter": max_iter,
    "guess_deviation":0,
    "guess_ratio":1.0,
    "mutation_strategy": 'DE/best/1'
}

configs = [random_config, random_fixedF_config, best_config, best_fixedF_config, taper_rand_config, taper_best_config]

In [ ]:
def run_plot_opts(function, dimensions, config_list, technical_replicates, name):
    fig, ax = plt.subplots(1, len(config_list), figsize=(5.5*len(config_list),6), sharey=True)
    fig.suptitle(name)
    opts = {}
    full_opts = {}
    for i, config in enumerate(config_list):
        opt_name = config['name']
        full_opts[opt_name] = []
        opt = (PSO_DE(function, n_dim=dimensions, config=config))
        opt.record_mode = True
        opt.run()
        print('i: best_x is ', opt.gbest_x, 'best_y is', opt.gbest_y)
        opts[opt_name] = opt
        full_opts[opt_name].append(opt)
        sum_loss = np.array(opt.gbest_y_hist)
        sum_x = np.array(opt.gbest_x)
        sum_y = opt.gbest_y
        for t in range(technical_replicates-1):
            opt = (PSO_DE(function, n_dim=dimensions, config=config))
            opt.record_mode = True
            opt.run()
            full_opts[opt_name].append(opt)
            sum_loss += opt.gbest_y_hist
            sum_x += opt.gbest_x
            sum_y += opt.gbest_y
        #fig.add_subplot(1, len(configs), i+1)
        average_loss = np.divide(sum_loss, technical_replicates)
        average_x = np.divide(sum_x, technical_replicates)
        average_y = sum_y / technical_replicates
        #average_x = ["{:.5e}".format(x) for x in average_x]
        results_text =  "{:.5e}".format(average_y[0]) + ' @ X: ' + str(average_x)
        ax[i].plot(average_loss)
        #plt.xlabel('Iteration')
        ax[i].set_ylabel('Best Known F(X)')
        ax[i].annotate(results_text, (0,0), (0,-20), xycoords='axes fraction', textcoords='offset points', va='top')
        ax[i].set_title(opt_name)

    fig.tight_layout()
    plt.show()

    return opts, full_opts

In [ ]:
def plot_swarm_w_loss(opts, config_list, title="Swarm Diversity & Loss"):
    Y_history = []
    fig, ax = plt.subplots(2, len(opts), figsize=(5*len(config_list),8), sharey='row')
    fig.suptitle(title)
    for i, key in enumerate(opts):
        opt = opts[key]
        Y_history = pd.DataFrame(np.array(opt.record_value['Y']).reshape((-1, opt.size_pop)))
        ax[0,i].plot(Y_history.index, Y_history.values, '.')
        ax[0,i].set_title(config_list[i]['name'])
        Y_history.min(axis=1).cummin().plot(kind='line', ax=ax[1,i])
        print(len(Y_history))
    plt.show()

Matyas  objective function

    Only takes two dimensions and has a global minimum at
    :code:`f([0,0])`. Its coordinates are bounded within
    :code:`[-10,10]`.

In [ ]:
import copy
n_dim = 2
lb=[-10., -10.]
ub = [10., 10.]
size_pop = 50
common_start = np.random.uniform(low=lb, high=ub, size=(size_pop, n_dim))

global_opt = (0, 0, 0)

In [ ]:
#Plot the sphere function's mesh for better plots
m = Mesher(func=example_funcs.matyas, limits=[(-10,10), (-10,10)])
#Adjust figure limits
d = Designer(limits=[(-10,10), (-10,10), (0,10)], label=['x-axis', 'y-axis', 'z-axis'])

In [ ]:
common_start = np.random.uniform(low=lb, high=ub, size=(size_pop, n_dim))

for config in configs:
    config["initial_guesses"] = common_start
    config["lb"] = lb
    config["ub"] = ub
    config["n_dim"] = n_dim
    config["vectorize_func"] = False


In [ ]:
opts, full_opts = run_plot_opts(example_funcs.matyas, n_dim, configs, technical_replicates, 'Matyas\nAverage Loss Over Optimization with '+ str(technical_replicates) + ' Technical Replicates')

In [ ]:
plot_swarm_w_loss(opts, configs, 'Matyas')

In [ ]:
%%capture
for i, opt in opts.items():
    animation2D = plot_contour(pos_history=opts[i].record_value['X'], mesher=m, designer=d, mark=global_opt)
    animation2D.save('matyas_'+str(i)+'.gif', writer='ffmpeg', fps=20)

In [ ]:
d.figsize= (len(configs) * 5.5, 15)
ani_summary = plot_summary(optimizers=list(opts.values()), title="Matyas", titles=[config['name'] for config in configs], mesher=m, designer=d, mark=global_opt)
plt.show()


In [ ]:
ani_summary.save('matyas_summary.gif', writer='ffmpeg', fps=15)
Image(url='matyas_summary.gif')

In [ ]:
%%capture
for i, opt in opts.items():
    pos_history_3d = m.compute_history_3d(opts[i].record_value['X']) #preprocessing
    animation3d = plot_surface(pos_history=pos_history_3d, mesher=m, designer=d, title="Matyas", mark=global_opt)
    animation3d.save('matyas_'+i+'_3d.gif', writer='ffmpeg', fps=15)
    print(i)

In [ ]:
Image('matyas_Random_3d.gif')

In [ ]:
%%capture
d.limits = [(-5,5), (-5,5), (-0.1, 100.0)]
for i, opt in opts.items():
    pos_history_3d = m.compute_history_3d(opts[i].record_value['X']) #preprocessing
    animation3d = plot_surface(pos_history=pos_history_3d, mesher=m, designer=d, title="Matyas", mark=global_opt)
    animation3d.save('matyas_zoom_'+i+'_3d.gif', writer='ffmpeg', fps=15)

Sphere  objective function

    Has a global minimum at :code:`0` and with a search domain of
        :code:`[-inf, inf]`

In [ ]:
n_dim = 2
lb = [-5., -5.]
ub = [5., 5.]
size_pop = 50
common_start = np.random.uniform(low=lb, high=ub, size=(size_pop, n_dim))

global_opt = (0, 0, 0)

In [ ]:
#Plot the sphere function's mesh for better plots
m = Mesher(func=example_funcs.ps_sphere)
#Adjust figure limits
d = Designer(limits=[(-1,1), (-1,1), (-0.1,1)], label=['x-axis', 'y-axis', 'z-axis'])

In [ ]:
common_start = np.random.uniform(low=lb, high=ub, size=(size_pop, n_dim))

for config in configs:
    config["initial_guesses"] = common_start
    config["lb"] = lb
    config["ub"] = ub
    config["n_dim"] = n_dim
    config["vectorize_func"] = True


In [ ]:
opts, full_opts = run_plot_opts(example_funcs.ps_sphere, n_dim, configs, technical_replicates, 'Sphere\nAverage Loss Over Optimization with '+ str(technical_replicates) + ' Technical Replicates')

In [ ]:
plot_swarm_w_loss(opts, configs, 'Sphere')

In [ ]:
%%capture
for i, opt in opts.items():
    animation2D = plot_contour(pos_history=opts[i].record_value['X'], mesher=m, designer=d, mark=global_opt)
    animation2D.save('sphere_'+str(i)+'.gif', writer='ffmpeg', fps=20)

In [ ]:
d.figsize= (len(configs) * 5.5, 15)
ani_summary = plot_summary(optimizers=list(opts.values()), title="Sphere", titles=[config['name'] for config in configs], mesher=m, designer=d, mark=global_opt)
plt.show()


In [ ]:
ani_summary.save('sphere_summary.gif', writer='ffmpeg', fps=15)
Image(url='sphere_summary.gif')

In [ ]:
%%capture
for i, opt in opts.items():
    pos_history_3d = m.compute_history_3d(opts[i].record_value['X']) #preprocessing
    animation3d = plot_surface(pos_history=pos_history_3d, mesher=m, designer=d, title="Sphere", mark=global_opt)
    animation3d.save('sphere_'+i+'_3d.gif', writer='ffmpeg', fps=15)
    print(i)

In [ ]:
Image('sphere_Random_3d.gif')

In [ ]:
%%capture
d.limits = [(-5,5), (-5,5), (-0.1, 100.0)]
for i, opt in opts.items():
    pos_history_3d = m.compute_history_3d(opts[i].record_value['X']) #preprocessing
    animation3d = plot_surface(pos_history=pos_history_3d, mesher=m, designer=d, title="Sphere", mark=global_opt)
    animation3d.save('sphere_zoom_'+i+'_3d.gif', writer='ffmpeg', fps=15)